# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus São Carlos</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Nome**: Fernando Messias da Silva - **RA**: 489450 </br>
**Nome**: Josie de Assis Francisco Henriques do Nascimento - **RA**: 840214


---
### Análise exploratória

Nesta seção, deve ser feita a leitura da base de dados e todas as análises necessárias para interpretar e analisar os dados, tais como:
* Significado de cada atributo
* Medidas descritivas
* Gráficos

In [ ]:
#Import Packages
import warnings
import numpy as np
import pandas as pd
import os

from matplotlib import pyplot as plt
import seaborn
from nltk import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV


warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Load csv file

#FILES_DIRECTORY = "/kaggle/input/ufscar-am-2024-projeto-final"
FILES_DIRECTORY = "dados"

rhp_data = pd.read_csv(os.path.join(FILES_DIRECTORY, "RHP_data.csv"), sep=',', encoding='utf-8')
train_data = pd.read_csv(os.path.join(FILES_DIRECTORY, "train.csv"), sep=',', index_col=None)
test_data = pd.read_csv(os.path.join(FILES_DIRECTORY, 'test.csv'), sep=',', index_col=None)
rhp_data_classe = pd.merge(rhp_data, train_data, on='Id')
#df_rhp_test = pd.merge(df_rhp, df_test, on='Id')


In [ ]:
rhp_data_classe

In [ ]:
# Cria um novo dataframe sem os atributos Idade, IMC, Atendimento, Convenio, Sexo, HDA1 e HDA2. Esses atributos são reduntantes ou não são relevantes para a análise a ser feito

columns_to_drop = ['DN', 'Atendimento', 'Convenio', 'HDA 1', 'HDA2']
rhp_data_processed = rhp_data_classe.drop(columns=columns_to_drop, errors='ignore')



In [ ]:
# Check the column structure of the dataset
rhp_data_processed.info()

In [ ]:
# Contar a distribuição das classes no conjunto de treino
class_distribution = rhp_data_processed["CLASSE"].value_counts(dropna=False)

# Visualizar a distribuição das classes
plt.figure(figsize=(6,4))
seaborn.barplot(x=class_distribution.index, y=class_distribution.values, palette="coolwarm")
plt.xlabel("Classe")
plt.ylabel("Quantidade")
plt.title("Distribuição das Classes no Conjunto de Treinamento")
plt.show()

In [ ]:
# Identifying outliers

# Histogram for each attribute:
for attribute in rhp_data_processed.columns:
    n, bins, patches = plt.hist(rhp_data_processed[attribute], bins=10, color='blue', alpha=0.7, rwidth=0.85)
    plt.title(f"Histograma do Atributo {attribute}")
    plt.show()

In [ ]:
# Density chart for each attribute (Help to identify outliers)
for attribute in rhp_data_processed.columns:
    densityPlot = rhp_data_processed[attribute].plot(kind='density', subplots=True, layout=(1, 1), sharex=False, sharey=False)
    plt.title(f"Gráfico de Densidade do Atributo {attribute}")
    plt.show()

In [ ]:
# Check for null values in the dataset
missing_values = rhp_data.isnull().sum().sort_values(ascending=False)

# Show attributes with higher count of null values
missing_values = missing_values[missing_values > 0]
missing_values

In [ ]:
# Análise Estatística
# Select only numeric columns
numerical_columns = rhp_data_processed.select_dtypes(include=["int64", "float64"]).columns

# Generate descriptive statistics
stats_summary = rhp_data_processed[numerical_columns].describe()

# Display the summary
display(stats_summary)

In [ ]:
# Distribution plot
# Main numeric variables plot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

seaborn.histplot(rhp_data_processed["Peso"].dropna(), bins=50, kde=True, ax=axes[0, 0])
axes[0, 0].set_title("Distribuição do Peso")

seaborn.histplot(rhp_data_processed["Altura"].dropna(), bins=50, kde=True, ax=axes[0, 1])
axes[0, 1].set_title("Distribuição da Altura")

seaborn.histplot(rhp_data_processed["IMC"].dropna(), bins=50, kde=True, ax=axes[1, 0])
axes[1, 0].set_title("Distribuição do IMC")

seaborn.histplot(rhp_data_processed["PA SISTOLICA"].dropna(), bins=50, kde=True, ax=axes[1, 1])
axes[1, 1].set_title("Distribuição da Pressão Sistólica")

plt.tight_layout()
plt.show()


In [ ]:
# Correlation Matrix
# Calcular a matriz de correlação entre as variáveis numéricas
correlation_matrix = rhp_data_processed[numerical_columns].corr()

# Gerar um mapa de calor para visualização das correlações
plt.figure(figsize=(10, 8))
seaborn.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Matriz de Correlação entre Variáveis Numéricas")
plt.show()


---
### Pré-processamento

Nesta seção, as funções da etapa de pré-processamento dos dados devem ser implementadas e aplicadas (se necessário).

In [ ]:
# Verificar a distribuição das classes no conjunto de treinamento
class_distribution = rhp_data_processed['CLASSE'].value_counts(normalize=True)

In [ ]:
class_distribution

In [ ]:
rhp_data_processed = rhp_data_processed.dropna(subset=['CLASSE'])

In [ ]:
pulsos_distribution = rhp_data_processed['PULSOS'].value_counts(normalize=True)
ppa_distribution = rhp_data_processed['PPA'].value_counts(normalize=True)
b2_distribution = rhp_data_processed['B2'].value_counts(normalize=True)
sopro_distribution = rhp_data_processed['SOPRO'].value_counts(normalize=True)
sexo_distribution = rhp_data_processed['SEXO'].value_counts(normalize=True)
motivo1_distribution = rhp_data_processed['MOTIVO1'].value_counts(normalize=True)
motivo2_distribution = rhp_data_processed['MOTIVO2'].value_counts(normalize=True)

In [ ]:
pulsos_distribution,ppa_distribution,b2_distribution,sopro_distribution,sexo_distribution,motivo1_distribution,motivo2_distribution

In [ ]:
# Replace #VALUE! to NaN
rhp_data_processed.replace("#VALUE!", pd.NA, inplace=True)

In [ ]:
rhp_data_processed['CLASSE'] = rhp_data_processed['CLASSE'].replace({
    'Normal': '0',
    'Normais': '0',
    'Anormal': '1'
}).astype('Int64')

In [ ]:
# Selecionar colunas categóricas para conversão
categorical_cols = ["SEXO", "PULSOS", "PPA", "B2", "SOPRO", "MOTIVO1", "MOTIVO2"]

# Aplicar One-Hot Encoding
rhp_data_processed = pd.get_dummies(rhp_data_processed, columns=categorical_cols, drop_first=True)

# Exibir as primeiras linhas após a codificação
rhp_data_processed.head()


In [ ]:
new_columns_to_drop = ['MOTIVO2_6 - Cianose e dispnéia', 'PULSOS_Amplos', 'PULSOS_NORMAIS', 'SOPRO_Sistolico e diastólico']
rhp_data_processed = rhp_data_processed.drop(columns=new_columns_to_drop, errors='ignore')

In [ ]:
rhp_data_processed.head()

In [ ]:
rhp_data_processed_normalized = rhp_data_processed
rhp_data_processed_normalized['Peso'] = pd.to_numeric(rhp_data_processed_normalized['Peso'], errors='coerce')
rhp_data_processed_normalized['Altura'] = pd.to_numeric(rhp_data_processed_normalized['Altura'], errors='coerce')
rhp_data_processed_normalized['IMC'] = pd.to_numeric(rhp_data_processed_normalized['IMC'], errors='coerce')
rhp_data_processed_normalized['IDADE'] = pd.to_numeric(rhp_data_processed_normalized['IDADE'], errors='coerce')
#rhp_data_processed_normalized['PULSOS'] = pd.to_numeric(rhp_data_processed_normalized['PULSOS'], errors='coerce')
rhp_data_processed_normalized['PA SISTOLICA'] = pd.to_numeric(rhp_data_processed_normalized['PA SISTOLICA'], errors='coerce')
rhp_data_processed_normalized['PA DIASTOLICA'] = pd.to_numeric(rhp_data_processed_normalized['PA DIASTOLICA'], errors='coerce')
#rhp_data_processed_normalized['PPA'] = pd.to_numeric(rhp_data_processed_normalized['PPA'], errors='coerce')
#rhp_data_processed_normalized['B2'] = pd.to_numeric(rhp_data_processed_normalized['B2'], errors='coerce')
rhp_data_processed_normalized['FC'] = pd.to_numeric(rhp_data_processed_normalized['FC'], errors='coerce')
# rhp_data_processed_normalized['SEXO'] = pd.to_numeric(rhp_data_processed_normalized['SEXO'], errors='coerce')
# rhp_data_processed_normalized['MOTIVO1'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO1'], errors='coerce')
# rhp_data_processed_normalized['MOTIVO2'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2'], errors='coerce')
# rhp_data_processed_normalized['SOPRO'] = pd.to_numeric(rhp_data_processed_normalized['SOPRO'], errors='coerce')
rhp_data_processed_normalized['SEXO_Feminino'] = pd.to_numeric(rhp_data_processed_normalized['SEXO_Feminino'], errors='coerce')
rhp_data_processed_normalized['SEXO_Indeterminado'] = pd.to_numeric(rhp_data_processed_normalized['SEXO_Indeterminado'], errors='coerce')
rhp_data_processed_normalized['SEXO_M'] = pd.to_numeric(rhp_data_processed_normalized['SEXO_M'], errors='coerce')
rhp_data_processed_normalized['SEXO_Masculino'] = pd.to_numeric(rhp_data_processed_normalized['SEXO_Masculino'], errors='coerce')
rhp_data_processed_normalized['SEXO_masculino'] = pd.to_numeric(rhp_data_processed_normalized['SEXO_masculino'], errors='coerce')
#rhp_data_processed_normalized['PULSOS_Amplos'] = pd.to_numeric(rhp_data_processed_normalized['PULSOS_Amplos'], errors='coerce')
rhp_data_processed_normalized['PULSOS_Diminuídos '] = pd.to_numeric(rhp_data_processed_normalized['PULSOS_Diminuídos '], errors='coerce')
rhp_data_processed_normalized['PULSOS_Femorais diminuidos'] = pd.to_numeric(rhp_data_processed_normalized['PULSOS_Femorais diminuidos'], errors='coerce')
#rhp_data_processed_normalized['PULSOS_NORMAIS'] = pd.to_numeric(rhp_data_processed_normalized['PULSOS_NORMAIS'], errors='coerce')
rhp_data_processed_normalized['PULSOS_Normais'] = pd.to_numeric(rhp_data_processed_normalized['PULSOS_Normais'], errors='coerce')
rhp_data_processed_normalized['PULSOS_Outro'] = pd.to_numeric(rhp_data_processed_normalized['PULSOS_Outro'], errors='coerce')
rhp_data_processed_normalized['PPA_HAS-1 PAS'] = pd.to_numeric(rhp_data_processed_normalized['PPA_HAS-1 PAS'], errors='coerce')
rhp_data_processed_normalized['PPA_HAS-2 PAD'] = pd.to_numeric(rhp_data_processed_normalized['PPA_HAS-2 PAD'], errors='coerce')
rhp_data_processed_normalized['PPA_HAS-2 PAS'] = pd.to_numeric(rhp_data_processed_normalized['PPA_HAS-2 PAS'], errors='coerce')
rhp_data_processed_normalized['PPA_Normal'] = pd.to_numeric(rhp_data_processed_normalized['PPA_Normal'], errors='coerce')
rhp_data_processed_normalized['PPA_Não Calculado'] = pd.to_numeric(rhp_data_processed_normalized['PPA_Não Calculado'], errors='coerce')
rhp_data_processed_normalized['PPA_Pre-Hipertensão PAD'] = pd.to_numeric(rhp_data_processed_normalized['PPA_Pre-Hipertensão PAD'], errors='coerce')
rhp_data_processed_normalized['PPA_Pre-Hipertensão PAS'] = pd.to_numeric(rhp_data_processed_normalized['PPA_Pre-Hipertensão PAS'], errors='coerce')
rhp_data_processed_normalized['B2_Hiperfonética'] = pd.to_numeric(rhp_data_processed_normalized['B2_Hiperfonética'], errors='coerce')
rhp_data_processed_normalized['B2_Normal'] = pd.to_numeric(rhp_data_processed_normalized['B2_Normal'], errors='coerce')
rhp_data_processed_normalized['B2_Outro'] = pd.to_numeric(rhp_data_processed_normalized['B2_Outro'], errors='coerce')
rhp_data_processed_normalized['B2_Única'] = pd.to_numeric(rhp_data_processed_normalized['B2_Única'], errors='coerce')
#rhp_data_processed_normalized['SOPRO_Sistolico e diastólico'] = pd.to_numeric(rhp_data_processed_normalized['SOPRO_Sistolico e diastólico'], errors='coerce')
rhp_data_processed_normalized['SOPRO_Sistólico'] = pd.to_numeric(rhp_data_processed_normalized['SOPRO_Sistólico'], errors='coerce')
rhp_data_processed_normalized['SOPRO_ausente'] = pd.to_numeric(rhp_data_processed_normalized['SOPRO_ausente'], errors='coerce')
rhp_data_processed_normalized['SOPRO_contínuo'] = pd.to_numeric(rhp_data_processed_normalized['SOPRO_contínuo'], errors='coerce')
rhp_data_processed_normalized['SOPRO_diastólico'] = pd.to_numeric(rhp_data_processed_normalized['SOPRO_diastólico'], errors='coerce')
rhp_data_processed_normalized['SOPRO_sistólico'] = pd.to_numeric(rhp_data_processed_normalized['SOPRO_sistólico'], errors='coerce')
rhp_data_processed_normalized['MOTIVO1_2 - Check-up'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO1_2 - Check-up'], errors='coerce')
rhp_data_processed_normalized['MOTIVO1_5 - Parecer cardiológico'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO1_5 - Parecer cardiológico'], errors='coerce')
rhp_data_processed_normalized['MOTIVO1_6 - Suspeita de cardiopatia'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO1_6 - Suspeita de cardiopatia'], errors='coerce')
rhp_data_processed_normalized['MOTIVO1_7 - Outro'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO1_7 - Outro'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_1 - Cardiopatia congenica'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_1 - Cardiopatia congenica'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_5 - Atividade física'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_5 - Atividade física'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_5 - Cirurgia'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_5 - Cirurgia'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_5 - Uso de cisaprida'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_5 - Uso de cisaprida'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Alterações de pulso/perfusão'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Alterações de pulso/perfusão'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Cansaço'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Cansaço'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Cardiopatia na familia'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Cardiopatia na familia'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Cianose'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Cianose'], errors='coerce')
#rhp_data_processed_normalized['MOTIVO2_6 - Cianose e dispnéia'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Cianose e dispnéia'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Dispnéia'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Dispnéia'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Dor precordial'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Dor precordial'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - HAS/dislipidemia/obesidade'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - HAS/dislipidemia/obesidade'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Palpitação/taquicardia/arritmia'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Palpitação/taquicardia/arritmia'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_6 - Sopro'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_6 - Sopro'], errors='coerce')
rhp_data_processed_normalized['MOTIVO2_Outro'] = pd.to_numeric(rhp_data_processed_normalized['MOTIVO2_Outro'], errors='coerce')
rhp_data_processed_normalized['CLASSE'] = pd.to_numeric(rhp_data_processed_normalized['CLASSE'], errors='coerce')


In [ ]:
# Criar Pressão de Pulso Arterial (PPA numérica)
rhp_data_processed_normalized["PPA_num"] = rhp_data_processed_normalized["PA SISTOLICA"] - rhp_data_processed_normalized["PA DIASTOLICA"]

In [ ]:
rhp_data_processed_normalized['Peso'].fillna(rhp_data_processed_normalized['Peso'].median(), inplace=True)
rhp_data_processed_normalized['Altura'].fillna(rhp_data_processed_normalized['Altura'].median(), inplace=True)
rhp_data_processed_normalized['IMC'].fillna(rhp_data_processed_normalized['IMC'].median(), inplace=True)
rhp_data_processed_normalized['IDADE'].fillna(rhp_data_processed_normalized['IDADE'].median(), inplace=True)
#rhp_data_processed_normalized['PULSOS'].fillna(rhp_data_processed_normalized['PULSOS'].mode()[0], inplace=True)
rhp_data_processed_normalized['PA SISTOLICA'].fillna(rhp_data_processed_normalized['PA SISTOLICA'].median(), inplace=True)
rhp_data_processed_normalized['PA DIASTOLICA'].fillna(rhp_data_processed_normalized['PA DIASTOLICA'].median(), inplace=True)
#rhp_data_processed_normalized['PPA'].fillna(rhp_data_processed_normalized['PPA'].mode()[0], inplace=True)
#rhp_data_processed_normalized['B2'].fillna(rhp_data_processed_normalized['B2'].mode()[0], inplace=True)
rhp_data_processed_normalized['FC'].fillna(rhp_data_processed_normalized['FC'].median(), inplace=True)
# rhp_data_processed_normalized['SEXO'].fillna(rhp_data_processed_normalized['SEXO'].mode()[0], inplace=True)
# rhp_data_processed_normalized['MOTIVO1'].fillna(rhp_data_processed_normalized['MOTIVO1'].mode()[0], inplace=True)
# rhp_data_processed_normalized['MOTIVO2'].fillna(rhp_data_processed_normalized['MOTIVO2'].mode()[0], inplace=True)
# rhp_data_processed_normalized['SOPRO'].fillna(rhp_data_processed_normalized['SOPRO'].mode()[0], inplace=True)

rhp_data_processed_normalized['SEXO_Feminino'].fillna(rhp_data_processed_normalized['SEXO_Feminino'].mode()[0], inplace=True)
rhp_data_processed_normalized['SEXO_Indeterminado'].fillna(rhp_data_processed_normalized['SEXO_Indeterminado'].mode()[0], inplace=True)
rhp_data_processed_normalized['SEXO_M'].fillna(rhp_data_processed_normalized['SEXO_M'].mode()[0], inplace=True)
rhp_data_processed_normalized['SEXO_Masculino'].fillna(rhp_data_processed_normalized['SEXO_Masculino'].mode()[0], inplace=True)
rhp_data_processed_normalized['SEXO_masculino'].fillna(rhp_data_processed_normalized['SEXO_masculino'].mode()[0], inplace=True)
#rhp_data_processed_normalized['PULSOS_Amplos'].fillna(rhp_data_processed_normalized['PULSOS_Amplos'].mode()[0], inplace=True)
rhp_data_processed_normalized['PULSOS_Diminuídos '].fillna(rhp_data_processed_normalized['PULSOS_Diminuídos '].mode()[0], inplace=True)
rhp_data_processed_normalized['PULSOS_Femorais diminuidos'].fillna(rhp_data_processed_normalized['PULSOS_Femorais diminuidos'].mode()[0], inplace=True)
#rhp_data_processed_normalized['PULSOS_NORMAIS'].fillna(rhp_data_processed_normalized['PULSOS_NORMAIS'].mode()[0], inplace=True)
rhp_data_processed_normalized['PULSOS_Normais'].fillna(rhp_data_processed_normalized['PULSOS_Normais'].mode()[0], inplace=True)
rhp_data_processed_normalized['PULSOS_Outro'].fillna(rhp_data_processed_normalized['PULSOS_Outro'].mode()[0], inplace=True)
rhp_data_processed_normalized['PPA_HAS-1 PAS'].fillna(rhp_data_processed_normalized['PPA_HAS-1 PAS'].mode()[0], inplace=True)
rhp_data_processed_normalized['PPA_HAS-2 PAD'].fillna(rhp_data_processed_normalized['PPA_HAS-2 PAD'].mode()[0], inplace=True)
rhp_data_processed_normalized['PPA_HAS-2 PAS'].fillna(rhp_data_processed_normalized['PPA_HAS-2 PAS'].mode()[0], inplace=True)
rhp_data_processed_normalized['PPA_Normal'].fillna(rhp_data_processed_normalized['PPA_Normal'].mode()[0], inplace=True)
rhp_data_processed_normalized['PPA_Não Calculado'].fillna(rhp_data_processed_normalized['PPA_Não Calculado'].mode()[0], inplace=True)
rhp_data_processed_normalized['PPA_Pre-Hipertensão PAD'].fillna(rhp_data_processed_normalized['PPA_Pre-Hipertensão PAD'].mode()[0], inplace=True)
rhp_data_processed_normalized['PPA_Pre-Hipertensão PAS'].fillna(rhp_data_processed_normalized['PPA_Pre-Hipertensão PAS'].mode()[0], inplace=True)
rhp_data_processed_normalized['B2_Hiperfonética'].fillna(rhp_data_processed_normalized['B2_Hiperfonética'].mode()[0], inplace=True)
rhp_data_processed_normalized['B2_Normal'].fillna(rhp_data_processed_normalized['B2_Normal'].mode()[0], inplace=True)
rhp_data_processed_normalized['B2_Outro'].fillna(rhp_data_processed_normalized['B2_Outro'].mode()[0], inplace=True)
rhp_data_processed_normalized['B2_Única'].fillna(rhp_data_processed_normalized['B2_Única'].mode()[0], inplace=True)
#rhp_data_processed_normalized['SOPRO_Sistolico e diastólico'].fillna(rhp_data_processed_normalized['SOPRO_Sistolico e diastólico'].mode()[0], inplace=True)
rhp_data_processed_normalized['SOPRO_Sistólico'].fillna(rhp_data_processed_normalized['SOPRO_Sistólico'].mode()[0], inplace=True)
rhp_data_processed_normalized['SOPRO_ausente'].fillna(rhp_data_processed_normalized['SOPRO_ausente'].mode()[0], inplace=True)
rhp_data_processed_normalized['SOPRO_contínuo'].fillna(rhp_data_processed_normalized['SOPRO_contínuo'].mode()[0], inplace=True)
rhp_data_processed_normalized['SOPRO_diastólico'].fillna(rhp_data_processed_normalized['SOPRO_diastólico'].mode()[0], inplace=True)
rhp_data_processed_normalized['SOPRO_sistólico'].fillna(rhp_data_processed_normalized['SOPRO_sistólico'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO1_2 - Check-up'].fillna(rhp_data_processed_normalized['MOTIVO1_2 - Check-up'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO1_5 - Parecer cardiológico'].fillna(rhp_data_processed_normalized['MOTIVO1_5 - Parecer cardiológico'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO1_6 - Suspeita de cardiopatia'].fillna(rhp_data_processed_normalized['MOTIVO1_6 - Suspeita de cardiopatia'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO1_7 - Outro'].fillna(rhp_data_processed_normalized['MOTIVO1_7 - Outro'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_1 - Cardiopatia congenica'].fillna(rhp_data_processed_normalized['MOTIVO2_1 - Cardiopatia congenica'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_5 - Atividade física'].fillna(rhp_data_processed_normalized['MOTIVO2_5 - Atividade física'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_5 - Cirurgia'].fillna(rhp_data_processed_normalized['MOTIVO2_5 - Cirurgia'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_5 - Uso de cisaprida'].fillna(rhp_data_processed_normalized['MOTIVO2_5 - Uso de cisaprida'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Alterações de pulso/perfusão'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Alterações de pulso/perfusão'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Cansaço'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Cansaço'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Cardiopatia na familia'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Cardiopatia na familia'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Cianose'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Cianose'].mode()[0], inplace=True)
#rhp_data_processed_normalized['MOTIVO2_6 - Cianose e dispnéia'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Cianose e dispnéia'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Dispnéia'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Dispnéia'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Dor precordial'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Dor precordial'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - HAS/dislipidemia/obesidade'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - HAS/dislipidemia/obesidade'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Palpitação/taquicardia/arritmia'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Palpitação/taquicardia/arritmia'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_6 - Sopro'].fillna(rhp_data_processed_normalized['MOTIVO2_6 - Sopro'].mode()[0], inplace=True)
rhp_data_processed_normalized['MOTIVO2_Outro'].fillna(rhp_data_processed_normalized['MOTIVO2_Outro'].mode()[0], inplace=True)
#rhp_data_processed_normalized['Peso_Altura_Ratio'].fillna(rhp_data_processed_normalized['Peso_Altura_Ratio'].median(), inplace=True)
rhp_data_processed_normalized['PPA_num'].fillna(rhp_data_processed_normalized['PPA_num'].mode()[0], inplace=True)


rhp_data_processed_normalized['CLASSE'].fillna(rhp_data_processed_normalized['CLASSE'].median(), inplace=True)

In [ ]:
# Aplicar a padronização (Z-score) aos atributos numéricos
numerical_columns = ["IDADE", "Peso", "Altura", "IMC", "PA SISTOLICA", "PA DIASTOLICA", "FC", "PPA_num"]

In [ ]:
# Calcular a média e desvio padrão de cada atributo
means = rhp_data_processed_normalized[numerical_columns].mean()
stds = rhp_data_processed_normalized[numerical_columns].std()

In [ ]:
# Aplicar a padronização
rhp_data_processed_normalized[numerical_columns] = (rhp_data_processed_normalized[numerical_columns] - means) / stds


In [ ]:
rhp_data_processed_normalized.head()

In [ ]:
# Salvar dados pre-processados
rhp_data_processed_normalized.to_csv('dados/rhp_data_processed_normalized.csv', index=False)

---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais padrões e testando diferentes modelos.

In [ ]:
# Cria o arquivo de treinamento
train_dataset = train_data.merge(rhp_data_processed_normalized, on="Id", how="left")

In [ ]:
train_dataset = train_dataset.drop(columns=["CLASSE_x"]).rename(columns={"CLASSE_y": "CLASSE"})

In [ ]:
# Criar nova feature: Relação Peso/Altura
#train_dataset["Peso_Altura_Ratio"] = train_dataset["Peso"] / train_dataset["Altura"]

# Criar Pressão de Pulso Arterial (PPA numérica)
train_dataset["PPA_num"] = train_dataset["PA SISTOLICA"] - train_dataset["PA DIASTOLICA"]

In [ ]:
train_dataset

In [ ]:
train_dataset = train_dataset.drop(columns=new_columns_to_drop, errors='ignore')

In [ ]:
# Remove NaN from train dataset
train_dataset['Peso'].fillna(train_dataset['Peso'].median(), inplace=True)
train_dataset['Altura'].fillna(train_dataset['Altura'].median(), inplace=True)
train_dataset['IMC'].fillna(train_dataset['IMC'].median(), inplace=True)
train_dataset['IDADE'].fillna(train_dataset['IDADE'].median(), inplace=True)
#train_dataset['PULSOS'].fillna(train_dataset['PULSOS'].mode()[0], inplace=True)
train_dataset['PA SISTOLICA'].fillna(train_dataset['PA SISTOLICA'].median(), inplace=True)
train_dataset['PA DIASTOLICA'].fillna(train_dataset['PA DIASTOLICA'].median(), inplace=True)
#train_dataset['PPA'].fillna(train_dataset['PPA'].mode()[0], inplace=True)
#train_dataset['B2'].fillna(train_dataset['B2'].mode()[0], inplace=True)
train_dataset['FC'].fillna(train_dataset['FC'].median(), inplace=True)
# train_dataset['SEXO'].fillna(train_dataset['SEXO'].mode()[0], inplace=True)
# train_dataset['MOTIVO1'].fillna(train_dataset['MOTIVO1'].mode()[0], inplace=True)
# train_dataset['MOTIVO2'].fillna(train_dataset['MOTIVO2'].median(), inplace=True)
# train_dataset['SOPRO'].fillna(train_dataset['SOPRO'].mode()[0], inplace=True)


train_dataset['SEXO_Feminino'].fillna(train_dataset['SEXO_Feminino'].mode()[0], inplace=True)
train_dataset['SEXO_Indeterminado'].fillna(train_dataset['SEXO_Indeterminado'].mode()[0], inplace=True)
train_dataset['SEXO_M'].fillna(train_dataset['SEXO_M'].mode()[0], inplace=True)
train_dataset['SEXO_Masculino'].fillna(train_dataset['SEXO_Masculino'].mode()[0], inplace=True)
train_dataset['SEXO_masculino'].fillna(train_dataset['SEXO_masculino'].mode()[0], inplace=True)
#train_dataset['PULSOS_Amplos'].fillna(train_dataset['PULSOS_Amplos'].mode()[0], inplace=True)
train_dataset['PULSOS_Diminuídos '].fillna(train_dataset['PULSOS_Diminuídos '].mode()[0], inplace=True)
train_dataset['PULSOS_Femorais diminuidos'].fillna(train_dataset['PULSOS_Femorais diminuidos'].mode()[0], inplace=True)
#train_dataset['PULSOS_NORMAIS'].fillna(train_dataset['PULSOS_NORMAIS'].mode()[0], inplace=True)
train_dataset['PULSOS_Normais'].fillna(train_dataset['PULSOS_Normais'].mode()[0], inplace=True)
train_dataset['PULSOS_Outro'].fillna(train_dataset['PULSOS_Outro'].mode()[0], inplace=True)
train_dataset['PPA_HAS-1 PAS'].fillna(train_dataset['PPA_HAS-1 PAS'].mode()[0], inplace=True)
train_dataset['PPA_HAS-2 PAD'].fillna(train_dataset['PPA_HAS-2 PAD'].mode()[0], inplace=True)
train_dataset['PPA_HAS-2 PAS'].fillna(train_dataset['PPA_HAS-2 PAS'].mode()[0], inplace=True)
train_dataset['PPA_Normal'].fillna(train_dataset['PPA_Normal'].mode()[0], inplace=True)
train_dataset['PPA_Não Calculado'].fillna(train_dataset['PPA_Não Calculado'].mode()[0], inplace=True)
train_dataset['PPA_Pre-Hipertensão PAD'].fillna(train_dataset['PPA_Pre-Hipertensão PAD'].mode()[0], inplace=True)
train_dataset['PPA_Pre-Hipertensão PAS'].fillna(train_dataset['PPA_Pre-Hipertensão PAS'].mode()[0], inplace=True)
train_dataset['B2_Hiperfonética'].fillna(train_dataset['B2_Hiperfonética'].mode()[0], inplace=True)
train_dataset['B2_Normal'].fillna(train_dataset['B2_Normal'].mode()[0], inplace=True)
train_dataset['B2_Outro'].fillna(train_dataset['B2_Outro'].mode()[0], inplace=True)
train_dataset['B2_Única'].fillna(train_dataset['B2_Única'].mode()[0], inplace=True)
#train_dataset['SOPRO_Sistolico e diastólico'].fillna(train_dataset['SOPRO_Sistolico e diastólico'].mode()[0], inplace=True)
train_dataset['SOPRO_Sistólico'].fillna(train_dataset['SOPRO_Sistólico'].mode()[0], inplace=True)
train_dataset['SOPRO_ausente'].fillna(train_dataset['SOPRO_ausente'].mode()[0], inplace=True)
train_dataset['SOPRO_contínuo'].fillna(train_dataset['SOPRO_contínuo'].mode()[0], inplace=True)
train_dataset['SOPRO_diastólico'].fillna(train_dataset['SOPRO_diastólico'].mode()[0], inplace=True)
train_dataset['SOPRO_sistólico'].fillna(train_dataset['SOPRO_sistólico'].mode()[0], inplace=True)
train_dataset['MOTIVO1_2 - Check-up'].fillna(train_dataset['MOTIVO1_2 - Check-up'].mode()[0], inplace=True)
train_dataset['MOTIVO1_5 - Parecer cardiológico'].fillna(train_dataset['MOTIVO1_5 - Parecer cardiológico'].mode()[0], inplace=True)
train_dataset['MOTIVO1_6 - Suspeita de cardiopatia'].fillna(train_dataset['MOTIVO1_6 - Suspeita de cardiopatia'].mode()[0], inplace=True)
train_dataset['MOTIVO1_7 - Outro'].fillna(train_dataset['MOTIVO1_7 - Outro'].mode()[0], inplace=True)
train_dataset['MOTIVO2_1 - Cardiopatia congenica'].fillna(train_dataset['MOTIVO2_1 - Cardiopatia congenica'].mode()[0], inplace=True)
train_dataset['MOTIVO2_5 - Atividade física'].fillna(train_dataset['MOTIVO2_5 - Atividade física'].mode()[0], inplace=True)
train_dataset['MOTIVO2_5 - Cirurgia'].fillna(train_dataset['MOTIVO2_5 - Cirurgia'].mode()[0], inplace=True)
train_dataset['MOTIVO2_5 - Uso de cisaprida'].fillna(train_dataset['MOTIVO2_5 - Uso de cisaprida'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Alterações de pulso/perfusão'].fillna(train_dataset['MOTIVO2_6 - Alterações de pulso/perfusão'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Cansaço'].fillna(train_dataset['MOTIVO2_6 - Cansaço'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Cardiopatia na familia'].fillna(train_dataset['MOTIVO2_6 - Cardiopatia na familia'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Cianose'].fillna(train_dataset['MOTIVO2_6 - Cianose'].mode()[0], inplace=True)
#train_dataset['MOTIVO2_6 - Cianose e dispnéia'].fillna(train_dataset['MOTIVO2_6 - Cianose e dispnéia'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Dispnéia'].fillna(train_dataset['MOTIVO2_6 - Dispnéia'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Dor precordial'].fillna(train_dataset['MOTIVO2_6 - Dor precordial'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - HAS/dislipidemia/obesidade'].fillna(train_dataset['MOTIVO2_6 - HAS/dislipidemia/obesidade'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Palpitação/taquicardia/arritmia'].fillna(train_dataset['MOTIVO2_6 - Palpitação/taquicardia/arritmia'].mode()[0], inplace=True)
train_dataset['MOTIVO2_6 - Sopro'].fillna(train_dataset['MOTIVO2_6 - Sopro'].mode()[0], inplace=True)
train_dataset['MOTIVO2_Outro'].fillna(train_dataset['MOTIVO2_Outro'].mode()[0], inplace=True)
#train_dataset['Peso_Altura_Ratio'].fillna(train_dataset['Peso_Altura_Ratio'].median(), inplace=True)
train_dataset['PPA_num'].fillna(train_dataset['PPA_num'].mode()[0], inplace=True)


train_dataset['CLASSE'].fillna(train_dataset['CLASSE'].median(), inplace=True)

In [ ]:
# Aplicar a padronização (Z-score) aos atributos numéricos
numerical_columns = ["IDADE", "Peso", "Altura", "IMC", "PA SISTOLICA", "PA DIASTOLICA", "FC", "PPA_num"]

In [ ]:
# Calcular a média e desvio padrão de cada atributo
means = train_dataset[numerical_columns].mean()
stds = train_dataset[numerical_columns].std()

In [ ]:
# Aplicar a padronização
train_dataset[numerical_columns] = (train_dataset[numerical_columns] - means) / stds

In [ ]:
train_dataset

In [ ]:
# Salvar dados pre-processados
train_dataset.to_csv('dados/train_dataset.csv', index=False)

In [ ]:
### Começar a aplicar os algoritmos para fazer o treinamento
X = train_dataset.drop(['CLASSE', 'Id'], axis=1)
X.fillna(0)
y = train_dataset['CLASSE']

In [ ]:
# Dividir em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Aplicar SMOTE para balancear as classes
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train.astype(int), y_train.astype(int))

In [ ]:
# Dicionário para armazenar resultados
results = {}

In [ ]:
# Treinar modelo
# k-NN
knn = KNeighborsClassifier(n_neighbors=15, weights='distance')
knn.fit(X_train_balanced, y_train_balanced)
y_pred_knn = knn.predict_proba(X_val)[:, 1]
results['k-NN'] = roc_auc_score(y_val, y_pred_knn)

In [ ]:
# Treinar modelo
# RandomForest
randomForest = RandomForestClassifier(n_estimators=750, max_depth=20, min_samples_split=5, random_state=42)
randomForest.fit(X_train_balanced, y_train_balanced)
y_pred_random_forest = randomForest.predict_proba(X_val)[:, 1]
results['Radom Forest'] = roc_auc_score(y_val, y_pred_random_forest)

In [ ]:
# Treinar modelo
# Logistic Regression
logisticRegression = LogisticRegression(max_iter=750)
logisticRegression.fit(X_train_balanced, y_train_balanced)
y_pred_logistic_regression = logisticRegression.predict_proba(X_val)[:, 1]
results['Logistic Regression'] = roc_auc_score(y_val, y_pred_logistic_regression)

In [ ]:
# Naive Bayes
nb = GaussianNB()
nb_calibrated = CalibratedClassifierCV(nb, method="sigmoid")
nb_calibrated.fit(X_train_balanced, y_train_balanced)
y_pred_nb = nb_calibrated.predict_proba(X_val)[:, 1]
results['Naive Bayes'] = roc_auc_score(y_val, y_pred_nb)

In [ ]:
# Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=750, learning_rate=0.05, max_depth=4, random_state=42)
gb.fit(X_train_balanced, y_train_balanced)
y_pred_gb = gb.predict_proba(X_val)[:, 1]
results['Gradient Boosting'] = roc_auc_score(y_val, y_pred_gb)

In [ ]:
# SVM
svm = SVC(probability=True, random_state=42)
svm.fit(X_train_balanced, y_train_balanced)
y_pred_svm = svm.predict_proba(X_val)[:, 1]
results['SVM'] = roc_auc_score(y_val, y_pred_svm)

In [ ]:
# Redes Neurais
mlp = MLPClassifier(random_state=42, max_iter=10000)
mlp.fit(X_train_balanced, y_train_balanced)
y_pred_mlp = mlp.predict_proba(X_val)[:, 1]
results['Redes Neurais'] = roc_auc_score(y_val, y_pred_mlp)

In [ ]:
# Definir o grid de hiperparâmetros
param_grid = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "criterion": ["gini", "entropy"],
    "max_features": ["sqrt", "log2", None],
    "class_weight": [None, "balanced"]
}

In [ ]:
# Decision Tree
dt = tree.DecisionTreeClassifier(criterion='gini', max_depth=3)
dt.fit(X_train_balanced, y_train_balanced)
y_pred_dt = dt.predict_proba(X_val)[:, 1]
results['Decision Tree'] = roc_auc_score(y_val, y_pred_dt)

In [ ]:
# Aplicar GridSearchCV
grid_search = GridSearchCV(dt, param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid_search.fit(X_train_balanced, y_train_balanced)
y_pred_gs = grid_search.predict_proba(X_val)[:, 1]
results['Grid Search'] = roc_auc_score(y_val, y_pred_gs)

In [ ]:
results

In [ ]:
# Filtrar os dados de teste baseados nos IDs do test.csv
test_filtered = test_data.merge(rhp_data, on="Id", how="left")

In [ ]:
# Remove columns dropped during training phase
test_filtered = test_filtered.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
# Selecionar colunas categóricas para conversão
categorical_cols = ["SEXO", "PULSOS", "PPA", "B2", "SOPRO", "MOTIVO1", "MOTIVO2"]

# Aplicar One-Hot Encoding
test_filtered = pd.get_dummies(test_filtered, columns=categorical_cols, drop_first=True)

# Exibir as primeiras linhas após a codificação
test_filtered.head()

In [ ]:
# Criar nova feature: Relação Peso/Altura
#test_filtered["Peso_Altura_Ratio"] = test_filtered["Peso"] / test_filtered["Altura"]

# Criar Pressão de Pulso Arterial (PPA numérica)
test_filtered["PPA_num"] = test_filtered["PA SISTOLICA"] - test_filtered["PA DIASTOLICA"]

In [ ]:
test_filtered

In [ ]:
test_filtered['Peso'] = pd.to_numeric(test_filtered['Peso'], errors='coerce')
test_filtered['Altura'] = pd.to_numeric(test_filtered['Altura'], errors='coerce')
test_filtered['IMC'] = pd.to_numeric(test_filtered['IMC'], errors='coerce')
test_filtered['IDADE'] = pd.to_numeric(test_filtered['IDADE'], errors='coerce')
#test_filtered['PULSOS'] = pd.to_numeric(test_filtered['PULSOS'], errors='coerce')
test_filtered['PA SISTOLICA'] = pd.to_numeric(test_filtered['PA SISTOLICA'], errors='coerce')
test_filtered['PA DIASTOLICA'] = pd.to_numeric(test_filtered['PA DIASTOLICA'], errors='coerce')
#test_filtered['PPA'] = pd.to_numeric(test_filtered['PPA'], errors='coerce')
#test_filtered['B2'] = pd.to_numeric(test_filtered['B2'], errors='coerce')
test_filtered['FC'] = pd.to_numeric(test_filtered['FC'], errors='coerce')
#test_filtered['SEXO'] = pd.to_numeric(test_filtered['SEXO'], errors='coerce')
#test_filtered['MOTIVO1'] = pd.to_numeric(test_filtered['MOTIVO1'], errors='coerce')
#test_filtered['MOTIVO2'] = pd.to_numeric(test_filtered['MOTIVO2'], errors='coerce')
#test_filtered['SOPRO'] = pd.to_numeric(test_filtered['SOPRO'], errors='coerce')

test_filtered['SEXO_Feminino'] = pd.to_numeric(test_filtered['SEXO_Feminino'], errors='coerce')
test_filtered['SEXO_Indeterminado'] = pd.to_numeric(test_filtered['SEXO_Indeterminado'], errors='coerce')
test_filtered['SEXO_M'] = pd.to_numeric(test_filtered['SEXO_M'], errors='coerce')
test_filtered['SEXO_Masculino'] = pd.to_numeric(test_filtered['SEXO_Masculino'], errors='coerce')
test_filtered['SEXO_masculino'] = pd.to_numeric(test_filtered['SEXO_masculino'], errors='coerce')
#test_filtered['PULSOS_Amplos'] = pd.to_numeric(test_filtered['PULSOS_Amplos'], errors='coerce')
test_filtered['PULSOS_Diminuídos '] = pd.to_numeric(test_filtered['PULSOS_Diminuídos '], errors='coerce')
test_filtered['PULSOS_Femorais diminuidos'] = pd.to_numeric(test_filtered['PULSOS_Femorais diminuidos'], errors='coerce')
#test_filtered['PULSOS_NORMAIS'] = pd.to_numeric(test_filtered['PULSOS_NORMAIS'], errors='coerce')
test_filtered['PULSOS_Normais'] = pd.to_numeric(test_filtered['PULSOS_Normais'], errors='coerce')
test_filtered['PULSOS_Outro'] = pd.to_numeric(test_filtered['PULSOS_Outro'], errors='coerce')
test_filtered['PPA_HAS-1 PAS'] = pd.to_numeric(test_filtered['PPA_HAS-1 PAS'], errors='coerce')
test_filtered['PPA_HAS-2 PAD'] = pd.to_numeric(test_filtered['PPA_HAS-2 PAD'], errors='coerce')
test_filtered['PPA_HAS-2 PAS'] = pd.to_numeric(test_filtered['PPA_HAS-2 PAS'], errors='coerce')
test_filtered['PPA_Normal'] = pd.to_numeric(test_filtered['PPA_Normal'], errors='coerce')
test_filtered['PPA_Não Calculado'] = pd.to_numeric(test_filtered['PPA_Não Calculado'], errors='coerce')
test_filtered['PPA_Pre-Hipertensão PAD'] = pd.to_numeric(test_filtered['PPA_Pre-Hipertensão PAD'], errors='coerce')
test_filtered['PPA_Pre-Hipertensão PAS'] = pd.to_numeric(test_filtered['PPA_Pre-Hipertensão PAS'], errors='coerce')
test_filtered['B2_Hiperfonética'] = pd.to_numeric(test_filtered['B2_Hiperfonética'], errors='coerce')
test_filtered['B2_Normal'] = pd.to_numeric(test_filtered['B2_Normal'], errors='coerce')
test_filtered['B2_Outro'] = pd.to_numeric(test_filtered['B2_Outro'], errors='coerce')
test_filtered['B2_Única'] = pd.to_numeric(test_filtered['B2_Única'], errors='coerce')
#test_filtered['SOPRO_Sistolico e diastólico'] = pd.to_numeric(test_filtered['SOPRO_Sistolico e diastólico'], errors='coerce')
test_filtered['SOPRO_Sistólico'] = pd.to_numeric(test_filtered['SOPRO_Sistólico'], errors='coerce')
test_filtered['SOPRO_ausente'] = pd.to_numeric(test_filtered['SOPRO_ausente'], errors='coerce')
test_filtered['SOPRO_contínuo'] = pd.to_numeric(test_filtered['SOPRO_contínuo'], errors='coerce')
test_filtered['SOPRO_diastólico'] = pd.to_numeric(test_filtered['SOPRO_diastólico'], errors='coerce')
test_filtered['SOPRO_sistólico'] = pd.to_numeric(test_filtered['SOPRO_sistólico'], errors='coerce')
test_filtered['MOTIVO1_2 - Check-up'] = pd.to_numeric(test_filtered['MOTIVO1_2 - Check-up'], errors='coerce')
test_filtered['MOTIVO1_5 - Parecer cardiológico'] = pd.to_numeric(test_filtered['MOTIVO1_5 - Parecer cardiológico'], errors='coerce')
test_filtered['MOTIVO1_6 - Suspeita de cardiopatia'] = pd.to_numeric(test_filtered['MOTIVO1_6 - Suspeita de cardiopatia'], errors='coerce')
test_filtered['MOTIVO1_7 - Outro'] = pd.to_numeric(test_filtered['MOTIVO1_7 - Outro'], errors='coerce')
test_filtered['MOTIVO2_1 - Cardiopatia congenica'] = pd.to_numeric(test_filtered['MOTIVO2_1 - Cardiopatia congenica'], errors='coerce')
test_filtered['MOTIVO2_5 - Atividade física'] = pd.to_numeric(test_filtered['MOTIVO2_5 - Atividade física'], errors='coerce')
test_filtered['MOTIVO2_5 - Cirurgia'] = pd.to_numeric(test_filtered['MOTIVO2_5 - Cirurgia'], errors='coerce')
test_filtered['MOTIVO2_5 - Uso de cisaprida'] = pd.to_numeric(test_filtered['MOTIVO2_5 - Uso de cisaprida'], errors='coerce')
test_filtered['MOTIVO2_6 - Alterações de pulso/perfusão'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Alterações de pulso/perfusão'], errors='coerce')
test_filtered['MOTIVO2_6 - Cansaço'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Cansaço'], errors='coerce')
test_filtered['MOTIVO2_6 - Cardiopatia na familia'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Cardiopatia na familia'], errors='coerce')
test_filtered['MOTIVO2_6 - Cianose'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Cianose'], errors='coerce')
#test_filtered['MOTIVO2_6 - Cianose e dispnéia'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Cianose e dispnéia'], errors='coerce')
test_filtered['MOTIVO2_6 - Dispnéia'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Dispnéia'], errors='coerce')
test_filtered['MOTIVO2_6 - Dor precordial'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Dor precordial'], errors='coerce')
test_filtered['MOTIVO2_6 - HAS/dislipidemia/obesidade'] = pd.to_numeric(test_filtered['MOTIVO2_6 - HAS/dislipidemia/obesidade'], errors='coerce')
test_filtered['MOTIVO2_6 - Palpitação/taquicardia/arritmia'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Palpitação/taquicardia/arritmia'], errors='coerce')
test_filtered['MOTIVO2_6 - Sopro'] = pd.to_numeric(test_filtered['MOTIVO2_6 - Sopro'], errors='coerce')
test_filtered['MOTIVO2_Outro'] = pd.to_numeric(test_filtered['MOTIVO2_Outro'], errors='coerce')




In [ ]:
test_filtered['Peso'].fillna(test_filtered['Peso'].median(), inplace=True)
test_filtered['Altura'].fillna(test_filtered['Altura'].median(), inplace=True)
test_filtered['IMC'].fillna(test_filtered['IMC'].median(), inplace=True)
test_filtered['IDADE'].fillna(test_filtered['IDADE'].median(), inplace=True)
#test_filtered['PULSOS'].fillna(test_filtered['PULSOS'].mode()[0], inplace=True)
test_filtered['PA SISTOLICA'].fillna(test_filtered['PA SISTOLICA'].median(), inplace=True)
test_filtered['PA DIASTOLICA'].fillna(test_filtered['PA DIASTOLICA'].median(), inplace=True)
#test_filtered['PPA'].fillna(test_filtered['PPA'].mode()[0], inplace=True)
#test_filtered['B2'].fillna(test_filtered['B2'].mode()[0], inplace=True)
test_filtered['FC'].fillna(test_filtered['FC'].median(), inplace=True)
#test_filtered['SEXO'].fillna(test_filtered['SEXO'].mode()[0], inplace=True)
#test_filtered['MOTIVO1'].fillna(test_filtered['MOTIVO1'].mode()[0], inplace=True)
#test_filtered['MOTIVO2'].fillna(test_filtered['MOTIVO2'].mode()[0], inplace=True)
#test_filtered['SOPRO'].fillna(test_filtered['SOPRO'].mode()[0], inplace=True)


test_filtered['SEXO_Feminino'].fillna(test_filtered['SEXO_Feminino'].mode()[0], inplace=True)
test_filtered['SEXO_Indeterminado'].fillna(test_filtered['SEXO_Indeterminado'].mode()[0], inplace=True)
test_filtered['SEXO_M'].fillna(test_filtered['SEXO_M'].mode()[0], inplace=True)
test_filtered['SEXO_Masculino'].fillna(test_filtered['SEXO_Masculino'].mode()[0], inplace=True)
test_filtered['SEXO_masculino'].fillna(test_filtered['SEXO_masculino'].mode()[0], inplace=True)
#test_filtered['PULSOS_Amplos'].fillna(test_filtered['PULSOS_Amplos'].mode()[0], inplace=True)
test_filtered['PULSOS_Diminuídos '].fillna(test_filtered['PULSOS_Diminuídos '].mode()[0], inplace=True)
test_filtered['PULSOS_Femorais diminuidos'].fillna(test_filtered['PULSOS_Femorais diminuidos'].mode()[0], inplace=True)
#test_filtered['PULSOS_NORMAIS'].fillna(test_filtered['PULSOS_NORMAIS'].mode()[0], inplace=True)
test_filtered['PULSOS_Normais'].fillna(test_filtered['PULSOS_Normais'].mode()[0], inplace=True)
test_filtered['PULSOS_Outro'].fillna(test_filtered['PULSOS_Outro'].mode()[0], inplace=True)
test_filtered['PPA_HAS-1 PAS'].fillna(test_filtered['PPA_HAS-1 PAS'].mode()[0], inplace=True)
test_filtered['PPA_HAS-2 PAD'].fillna(test_filtered['PPA_HAS-2 PAD'].mode()[0], inplace=True)
test_filtered['PPA_HAS-2 PAS'].fillna(test_filtered['PPA_HAS-2 PAS'].mode()[0], inplace=True)
test_filtered['PPA_Normal'].fillna(test_filtered['PPA_Normal'].mode()[0], inplace=True)
test_filtered['PPA_Não Calculado'].fillna(test_filtered['PPA_Não Calculado'].mode()[0], inplace=True)
test_filtered['PPA_Pre-Hipertensão PAD'].fillna(test_filtered['PPA_Pre-Hipertensão PAD'].mode()[0], inplace=True)
test_filtered['PPA_Pre-Hipertensão PAS'].fillna(test_filtered['PPA_Pre-Hipertensão PAS'].mode()[0], inplace=True)
test_filtered['B2_Hiperfonética'].fillna(test_filtered['B2_Hiperfonética'].mode()[0], inplace=True)
test_filtered['B2_Normal'].fillna(test_filtered['B2_Normal'].mode()[0], inplace=True)
test_filtered['B2_Outro'].fillna(test_filtered['B2_Outro'].mode()[0], inplace=True)
test_filtered['B2_Única'].fillna(test_filtered['B2_Única'].mode()[0], inplace=True)
#test_filtered['SOPRO_Sistolico e diastólico'].fillna(test_filtered['SOPRO_Sistolico e diastólico'].mode()[0], inplace=True)
test_filtered['SOPRO_Sistólico'].fillna(test_filtered['SOPRO_Sistólico'].mode()[0], inplace=True)
test_filtered['SOPRO_ausente'].fillna(test_filtered['SOPRO_ausente'].mode()[0], inplace=True)
test_filtered['SOPRO_contínuo'].fillna(test_filtered['SOPRO_contínuo'].mode()[0], inplace=True)
test_filtered['SOPRO_diastólico'].fillna(test_filtered['SOPRO_diastólico'].mode()[0], inplace=True)
test_filtered['SOPRO_sistólico'].fillna(test_filtered['SOPRO_sistólico'].mode()[0], inplace=True)
test_filtered['MOTIVO1_2 - Check-up'].fillna(test_filtered['MOTIVO1_2 - Check-up'].mode()[0], inplace=True)
test_filtered['MOTIVO1_5 - Parecer cardiológico'].fillna(test_filtered['MOTIVO1_5 - Parecer cardiológico'].mode()[0], inplace=True)
test_filtered['MOTIVO1_6 - Suspeita de cardiopatia'].fillna(test_filtered['MOTIVO1_6 - Suspeita de cardiopatia'].mode()[0], inplace=True)
test_filtered['MOTIVO1_7 - Outro'].fillna(test_filtered['MOTIVO1_7 - Outro'].mode()[0], inplace=True)
test_filtered['MOTIVO2_1 - Cardiopatia congenica'].fillna(test_filtered['MOTIVO2_1 - Cardiopatia congenica'].mode()[0], inplace=True)
test_filtered['MOTIVO2_5 - Atividade física'].fillna(test_filtered['MOTIVO2_5 - Atividade física'].mode()[0], inplace=True)
test_filtered['MOTIVO2_5 - Cirurgia'].fillna(test_filtered['MOTIVO2_5 - Cirurgia'].mode()[0], inplace=True)
test_filtered['MOTIVO2_5 - Uso de cisaprida'].fillna(test_filtered['MOTIVO2_5 - Uso de cisaprida'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Alterações de pulso/perfusão'].fillna(test_filtered['MOTIVO2_6 - Alterações de pulso/perfusão'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Cansaço'].fillna(test_filtered['MOTIVO2_6 - Cansaço'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Cardiopatia na familia'].fillna(test_filtered['MOTIVO2_6 - Cardiopatia na familia'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Cianose'].fillna(test_filtered['MOTIVO2_6 - Cianose'].mode()[0], inplace=True)
#test_filtered['MOTIVO2_6 - Cianose e dispnéia'].fillna(test_filtered['MOTIVO2_6 - Cianose e dispnéia'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Dispnéia'].fillna(test_filtered['MOTIVO2_6 - Dispnéia'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Dor precordial'].fillna(test_filtered['MOTIVO2_6 - Dor precordial'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - HAS/dislipidemia/obesidade'].fillna(test_filtered['MOTIVO2_6 - HAS/dislipidemia/obesidade'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Palpitação/taquicardia/arritmia'].fillna(test_filtered['MOTIVO2_6 - Palpitação/taquicardia/arritmia'].mode()[0], inplace=True)
test_filtered['MOTIVO2_6 - Sopro'].fillna(test_filtered['MOTIVO2_6 - Sopro'].mode()[0], inplace=True)
test_filtered['MOTIVO2_Outro'].fillna(test_filtered['MOTIVO2_Outro'].mode()[0], inplace=True)
#test_filtered['Peso_Altura_Ratio'].fillna(train_dataset['Peso_Altura_Ratio'].median(), inplace=True)
test_filtered['PPA_num'].fillna(test_filtered['PPA_num'].mode()[0], inplace=True)



In [ ]:
# Aplicar a padronização (Z-score) aos atributos numéricos
numerical_columns = ["IDADE", "Peso", "Altura", "IMC", "PA SISTOLICA", "PA DIASTOLICA", "FC", "PPA_num"]

In [ ]:
# Calcular a média e desvio padrão de cada atributo
means = test_filtered[numerical_columns].mean()
stds = test_filtered[numerical_columns].std()

In [ ]:
# Aplicar a padronização
test_filtered[numerical_columns] = (test_filtered[numerical_columns] - means) / stds

In [ ]:
# Remover a coluna CLASSE do conjunto de teste, pois essa é a variável a ser predita
if "CLASSE" in test_filtered.columns:
    test_filtered = test_filtered.drop(columns=["CLASSE"])

In [ ]:
# Separar features (X) e rótulos (y) do conjunto de treinamento
X_train = train_data.drop(columns=["Id", "CLASSE"])
y_train = train_data["CLASSE"]

In [ ]:
# Features do conjunto de teste
X_test = test_filtered.drop(columns=["Id"])

In [ ]:
# Verificar as dimensões dos conjuntos
X_train.shape, y_train.shape, X_test.shape

In [ ]:
# Fazer previsões no conjunto de teste
pred_knn = knn.predict_proba(X_test)
pred_nb = nb_calibrated.predict_proba(X_test)
pred_svm = svm.predict_proba(X_test)
pred_mlp = mlp.predict_proba(X_test)
pred_rf = randomForest.predict_proba(X_test)
pred_lr = logisticRegression.predict_proba(X_test)
pred_gb = gb.predict_proba(X_test)
pred_dt = dt.predict_proba(X_test)

In [ ]:
pred_knn

In [ ]:
# Criar um DataFrame para armazenar os resultados
results = test_filtered[["Id"]].copy()
results["k-NN"] = pred_knn[:, 1]
results["Naive Bayes"] = pred_nb[:, 1]
results["SVM"] = pred_svm[:, 1]
results["Redes Neurais"] = pred_mlp[:, 1]
results["Random Forest"] = pred_rf[:, 1]
results["Logistic Regression"] = pred_lr[:, 1]
results["Gradient Boosting"] = pred_gb[:, 1]
results["Decision Tree"] = pred_dt[:, 1]

In [ ]:
results

In [ ]:
# Gera os arquivos de submissão

# Criar DataFrame para submissão
submission_naive_bayes = pd.DataFrame({
    'Id': results["Id"],
    'Predicted': results["Naive Bayes"]
})

submission_knn = pd.DataFrame({
    'Id': results["Id"],
    'Predicted': results["k-NN"]
})

submission_redes_neurais = pd.DataFrame({
    'Id': results["Id"],
    'Predicted': results["Redes Neurais"]
})

submission_random_forest = pd.DataFrame({
    'Id': results["Id"],
    'Predicted': results["Random Forest"]
})

submission_logistic_regression = pd.DataFrame({
    'Id': results["Id"],
    'Predicted': results["Logistic Regression"]
})

submission_gradient_boosting = pd.DataFrame({
    'Id': results["Id"],
    'Predicted': results["Gradient Boosting"]
})

submission_decision_tree = pd.DataFrame({
    'Id': results["Id"],
    'Predicted': results["Decision Tree"]
})

In [ ]:
# Salvar o arquivo
submission_naive_bayes.to_csv("dados/submission_naive_bayes.csv", index=False)
submission_redes_neurais.to_csv("dados/submission_redes_neurais.csv", index=False)
submission_knn.to_csv("dados/submission_knn.csv", index=False)
submission_random_forest.to_csv("dados/random_forest.csv", index=False)
submission_logistic_regression.to_csv("dados/logistic_regression.csv", index=False)
submission_gradient_boosting.to_csv("dados/gradient_boosting.csv", index=False)
submission_decision_tree.to_csv("dados/decision_tree.csv", index=False)

---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos através de tabelas e gráficos, comparados e profundamente analisados.